## [Demo] Working with PSANA inside Jupyter lab notebook

This notebook starts a Dask scheduler that serves as a bridge to share data between PSANA and Jupyter Lab. Using the `slurm` package, users can submit a PSANA job and return results to be manipulated within the notebook. 

This notebook is able to run the example shown in https://confluence.slac.stanford.edu/display/PSDMInternal/Small+H5

Note: `slurm_magic` version from https://github.com/muammar/slurm-magic. 

We write the python file used to run the job.

In [1]:
example = """from psana import DataSource
import numpy as np
import sys
from dask.distributed import Client, Queue

# The code below fail with MPI rank errors. 
# import os
# os.environ['PS_SRV_NODES'] = "2"

# Connect to the Dask scheduler
client = Client(scheduler_file="scheduler.json")

# Create a Queue object
queue = Queue("psana")

# called back on each SRV node, for every smd.event() call below
def test_callback(data_dict):
    print(data_dict)

ds = DataSource(exp='xpptut13', run=1, dir='.tmp')

# batch_size here specifies how often the dictionary of information
# is sent to the SRV nodes
smd = ds.smalldata(filename='my.h5', batch_size=5, callbacks=[test_callback])
run = next(ds.runs())

# necessary (instead of "None") since some ranks may not receive events
# and the smd.sum() below could fail

arrsum = np.zeros((2), dtype=np.int)

for i,evt in enumerate(run.events()):
    myones = np.ones_like(arrsum)
    smd.event(evt, myfloat=2.0, arrint=myones)
    arrsum += myones
    queue.put(arrsum.tolist())


# This fails as reported in https://github.com/slac-lcls/lcls2/issues/11

# if smd.summary:
#     smd.sum(arrsum)
#     smd.save_summary({'summary_array' : arrsum}, summary_int=1)
# smd.done()

sys.exit()
"""

with open("example.py", "w") as f:
    f.write(example)
    
f.close()

In [2]:
sub_file = """#!/bin/bash
#SBATCH -N 1
#SBATCH -n 3
#SBATCH -c 1
#SBATCH -C haswell
#SBATCH -q debug
#SBATCH -J smallD
#SBATCH --account=m3384
#SBATCH --mail-user=melkhatibr@lbl.gov
#SBATCH --mail-type=ALL
#SBATCH -t 00:30:00

export PS_SRV_NODES=2

srun -n 6 -o example.log python example.py
"""

with open("example.sub", "w") as f:
    f.write(sub_file)
    
f.close()

In [3]:
print("remove # to clean")
!rm -rf *.json *.log .tmp *.out dask-worker-space/

remove # to clean


Load the `slurm_magic` package

In [4]:
%load_ext slurm_magic
%squeue -u melkhati

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)


We proceed to start a scheduler in the login node to be our bridge to share data. 

In [5]:
from dask.distributed import Client, LocalCluster, Queue

In [6]:
scheduler_file = 'scheduler.json'
cluster = LocalCluster(n_workers=1, threads_per_worker=1, host="0.0.0.0")
client = Client(cluster)
client.write_scheduler_file(scheduler_file)

We can now verify our scheduler is up and running. We also created a `scheduler.json` file that can be used to connect as many notebooks as desired.

In [7]:
client

Client Scheduler: tcp://128.55.224.44:38879 Dashboard: http://128.55.224.44:8787/status,Cluster Workers: 1 Cores: 1 Memory: 540.15 GB


In [8]:
import os 

if os.path.isfile(scheduler_file):
    print("Scheduler file with name {} exists!".format(scheduler_file))

Scheduler file with name scheduler.json exists!


Now that client is up and running, there are potentially two ways to submit and gather data from a PSANA job:

1. Using a `Queue` in Dask. 
2. Using Pub/Sub scheme. 

## Queue example

In [9]:
!python /global/common/software/lcls/psana/lcls2/psana/psana/tests/setup_input_files.py


Start writing offsets.
Finished writing smd for 19 events with size (B): 39136

Start writing offsets.
Finished writing smd for 19 events with size (B): 34744


In [10]:
# %env PS_SRV_NODES=2
# %srun -n 3 -A m3384  -C haswell -J smallD -q debug  -o example.log python example.py

In [11]:
%sbatch example.sub

'Submitted batch job 33352641\n'

In [19]:
%squeue -u melkhati

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)
0,33352641,debug_hsw,smallD,melkhati,PD,0:00,1,(Priority)


In [20]:
q = Queue("psana")

In [21]:
q.qsize()

10

In [22]:
import numpy as np

results = []
for data in range(q.qsize()):
    results.append(np.array(q.get()))

In [23]:
results = np.array(results)
results

array([[1, 1],
       [1, 1],
       [2, 2],
       [2, 2],
       [3, 3],
       [3, 3],
       [4, 4],
       [4, 4],
       [5, 5],
       [5, 5]])